# 데이터 시각화

데이터 시각화를 하기 위해 필요한 패키지를 import 한다.

In [24]:
from plotly import tools, subplots 
import plotly.offline as py 
py.init_notebook_mode(connected=True) 
import plotly.graph_objs as go 
import plotly.express as px 
import plotly.figure_factory as ff 
import plotly.io as pio
pio.templates.default = "plotly_dark" 
import matplotlib.pyplot as plt 

위에서 만들엇던 데이터셋 ww_melt_df를 이용해서 그래프를 그려보자.

In [25]:
fig = px.line(ww_melt_df, x="date", y="value", color='variable',
              title="Worldwide Confirmed/Death Cases Over Time") 
fig.show()



Log scale을 이용한 지표

In [27]:
fig = px.line(ww_melt_df, x="date", y="value", color="variable",
                title="Worldwid Confirmed/Death Cases Over Time (Log scale)",
             log_y=True)
fig.show()

사망률을 토대로 그래프를 그려보자.

In [28]:
ww_df['mortality'] = ww_df['fatalities'] / ww_df['confirmed'] 
fig = px.line(ww_df, x="date", y="mortality", title="Worldwide Mortality Rate Over Time")
fig.show()



초기에 사망률이 높아졌다가 점차적으로 안정을 찾는 모습을 확인 할 수 있다.

Growth Factor 그래프를 그려보자.

In [29]:
fig = px.line(ww_df, x="date", y="growth_factor", 
              title="Worldwide Growth Factor Over Time")
fig.add_trace(go.Scatter(x=[ww_df['date'].min(), ww_df['date'].max()], y=[1., 1.], name='Growth factor=1.', line=dict(dash='dash', color=('rgb(255, 0, 0)')))) 
fig.update_yaxes(range=[0., 5.])
fig.show()



# 국가별 코로나 분석


국가별로 코로나19의 추이가 어떻게 되는지를 확인해 보자.<br>
우선 국가별 사망자와 진단 데이터를 생성한다.

In [30]:
country_df = train.groupby(['date','country'])[['confirmed', 'fatalities']].sum().reset_index()
country_df.tail()

,date,country,confirmed,fatalities
128179,2021-11-05,Vietnam,953547,22412
128180,2021-11-05,West Bank and Gaza,455099,4705
128181,2021-11-05,Yemen,9843,1905
128182,2021-11-05,Zambia,209852,3662
128183,2021-11-05,Zimbabwe,133112,4685


유니크한 국가명을 저장한다.

In [31]:
countries = country_df['country'].unique()
print(f'{len(countries)} countries are in dataset:\n{countries}')



196 countries are in dataset:
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Comoros' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel'

196개의 국가가 있음을 확인할 수 있다.

In [32]:
len(countries)

196

각 국가들이 얼마나 많은 진단을 하고 있는지 확인해본다.<br>
confirmed 데이터를 활용한다.

In [33]:
target_date = country_df['date'].max() 




confirmed값이 1000 이상인 국가들을 가지고 탑 다운 차트를 그려보자.

In [34]:
top_country_df = country_df.query('(date == @target_date) & (confirmed > 1000)').sort_values('confirmed', ascending=False) 
top_country_melt_df = pd.melt(top_country_df, id_vars='country', value_vars=['confirmed', 'fatalities'])




In [35]:
fig = px.bar(top_country_melt_df.iloc[::-1],
             x='value', y='country', color='variable', barmode='group',
             title=f'Confirmed Cases/Deaths on {target_date}', 
             text='value', height=3000, orientation='h') 
fig.show()



TOp30 국가들의 진단 데이터를 차트로 그려보자.

In [36]:
top30_countries = top_country_df.sort_values('confirmed', ascending=False).iloc[:30]['country'].unique() 
top30_countries_df = country_df[country_df['country'].isin(top30_countries)] 
fig = px.line(top30_countries_df, x='date', y='confirmed', color='country', title=f'Confirmed Cases for top 30 country as of {target_date}') 
fig.show()



유럽과 남미 국가들이 많이보이는 경향이 있다.

사망자 데이터를 이용해서도 그려본다.

In [37]:
top30_countries = top_country_df.sort_values('confirmed', ascending=False).iloc[:30]['country'].unique() 
kor_countries_df = country_df[country_df['country']=='Korea, South'] 
fig = px.line(kor_countries_df, x='date', y='confirmed', color='country', title=f'Confirmed Cases for top 30 country as of {target_date}') 
fig.show()



사망률을 가지고 데이터 시각화를 진행하고 데이터를 새로 만들어준다.

In [38]:
top_country_df = country_df.query('(date == @target_date) & (confirmed > 100)')
top_country_df['mortality_rate'] = top_country_df['fatalities'] / top_country_df['confirmed']
top_country_df = top_country_df.sort_values('mortality_rate', ascending=False)



사망률이 적은 30위 나라의 지표

In [39]:
fig = px.bar(top_country_df[-30:], x='mortality_rate', y='country', title=f'Mortality rate LOW: top 30 countries on {target_date}', text='mortality_rate', height=800, orientation='h') 
fig.show()



한국이 사망률 적은 30위인 것을 알 수 있다.

총 순위

In [40]:
fig = px.bar(top_country_df, x='mortality_rate', y='country', title=f'Mortality rate LOW: top 30 countries on {target_date}', text='mortality_rate', height=800, orientation='h') 
fig.show()



사망률에서는 예메인 가장 높은 것을 보여준다.<br>
별개로 의료수준이나 대응력을 평가 할만한 수치이다.